In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
! kaggle datasets download -d utkarshsaxenadn/car-vs-bike-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/utkarshsaxenadn/car-vs-bike-classification-dataset
License(s): CC0-1.0


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/car-vs-bike-classification-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [ ]:
import os
print(os.listdir("/content/Car-Bike-Dataset"))

['Bike', 'Car']


In [ ]:
train_dataset = keras.preprocessing.image_dataset_from_directory(
    directory='/content/Car-Bike-Dataset',
    labels='inferred',
    batch_size=32,
    image_size=(256, 256),
    label_mode='int',
    validation_split=0.2,
    subset='training',
    seed=123,
    shuffle=True
)


Found 4000 files belonging to 2 classes.
Using 3200 files for training.


In [ ]:
validation_dataset = keras.preprocessing.image_dataset_from_directory(
    directory='/content/Car-Bike-Dataset',
    labels='inferred',
    batch_size=32,
    image_size=(256, 256),
    label_mode='int',
    validation_split=0.2,
    subset='validation',
    seed=123,
    shuffle=True  # Ensure shuffling
)

Found 4000 files belonging to 2 classes.
Using 800 files for validation.


In [ ]:
def process(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(process).prefetch(tf.data.AUTOTUNE)
validation_dataset = validation_dataset.map(process).prefetch(tf.data.AUTOTUNE)


In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
from PIL import Image

folder_path = "/content/Car-Bike-Dataset"
output_format = "JPEG"

for root, _, files in os.walk(folder_path):
    for filename in files:
        file_path = os.path.join(root, filename)
        try:
            with Image.open(file_path) as img:
                rgb_img = img.convert("RGB")
                new_path = file_path.rsplit(".", 1)[0] + ".jpg"
                rgb_img.save(new_path, output_format)
                print(f"✅ Converted: {new_path}")
        except Exception as e:
            print(f"❌ Error with file: {file_path} -> {e}")


✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (54).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1453).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (182).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (187).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1346).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/images (51).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1149).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1266).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/images (104).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1450).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (696).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (488).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1095).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1529).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (86).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (63).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike 

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


✅ Converted: /content/Car-Bike-Dataset/Bike/image (16).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (8).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1387).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1128).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1015).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1066).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/images (29).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1203).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (577).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1220).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (263).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (713).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (108).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (234).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (490).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (1081).jpg
✅ Converted: /content/Car-Bike-Dataset/Bike/Bike (

In [ ]:
model.fit(train_dataset, epochs=10, validation_data=validation_dataset)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 88ms/step - accuracy: 0.6543 - loss: 0.7677 - val_accuracy: 0.9125 - val_loss: 0.2355
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 99ms/step - accuracy: 0.9112 - loss: 0.2335 - val_accuracy: 0.9438 - val_loss: 0.1680
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 96ms/step - accuracy: 0.9482 - loss: 0.1227 - val_accuracy: 0.9125 - val_loss: 0.2304
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 84ms/step - accuracy: 0.9664 - loss: 0.0826 - val_accuracy: 0.9287 - val_loss: 0.2007
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.9834 - loss: 0.0483 - val_accuracy: 0.9275 - val_loss: 0.2429
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 80ms/step - accuracy: 0.9875 - loss: 0.0350 - val_accuracy: 0.9262 - val_loss: 0.2652
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 85ms/step - accuracy: 0.9857 - loss: 0.0365 - val_accuracy: 0.9262 - val_loss: 0.3112
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/step - accuracy: 0.9891 - loss: 0.0326 - val_

In [ ]:
import cv2
test_image = cv2.imread('/content/car.jpg')

In [ ]:
if test_image is not None:
    test_image = cv2.resize(test_image, (256, 256))
else:
    print("Error: Image loading failed.")

In [ ]:
test_input = test_image.reshape((1,256,256,3))
test_input.shape

(1, 256, 256, 3)

In [ ]:
model.predict(test_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 731ms/step


array([[1.]], dtype=float32)

In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.2 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st
import tensorflow as tf
from PIL import Image
import numpy as np

model = tf.keras.models.load_model('/content/car_bike_model.h5')

class_labels = ["Car", "Bike"]

st.title("🚗🏍️ Car vs Bike Classifier")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file).convert("RGB")
    st.image(image, caption="Uploaded Image", use_column_width=True)

    image = image.resize((256, 256))
    image_array = np.array(image) / 255.0
    image_array = np.expand_dims(image_array, axis=0)

    prediction = model.predict(image_array)
    predicted_class = class_labels[np.argmax(prediction)]

    st.write(f"### Prediction: {predicted_class}")

Writing app.py


In [ ]:
!curl https://loca.lt/mytunnelpassword

34.16.190.188

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.190.188:8501

⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://tasty-banks-enter.loca.lt
2025-04-07 04:52:18.161928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744001538.186298    2026 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744001538.193728    2026 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-07 04:52:21.869414: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overri